<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data : https://data.stackexchange.com/stackoverflow/query/edit/1086274
    - Source : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse
    - Entrainement
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 06/08/2019 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Data</p>

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I.1 - Source</p>

Revoir les explications

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I.2 - Preparation des données</p>

In [1]:
import sys
import os
sys.path.insert(0, './Python')

try:
    from TagText import TagText
except Exception, e:
    print 'Import failed:', e


./Python\TagText.py:1: SyntaxWarning: import * only allowed at module level
  class TagText:


In [2]:
tagText = TagText()
print(tagText.urlDirectory)
print(tagText.fileName)
data_question = tagText.read_source()
data_question.head()

Data/
QuestionVsTags.csv


,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [3]:
print(tagText.stopwords)
print(tagText.badwords)

set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u'they', u'not', u'during', u'now', u'him', u'nor', u"wasn't", u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u"won't", u'where', u"mustn't", u"isn't", u'few', u'because', u"you'd", u'doing', u'some', u'hasn', u"hasn't", u'are', u'our', u'ourselves', u'out', u'what', u'for', u"needn't", u'below', u're', u'does', u"shouldn't", u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u"mightn't", u"doesn't", u'were', u'here', u'shouldn', u'hers', u"aren't", u'by', u'on', u'about', u'couldn', u'of', u"wouldn't", u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u"hadn't", u'mightn', u"couldn't", u'wasn', u'your', u"you're", u'from', u'her', u'their', u'aren', u"it's",

In [4]:
import numpy as np
import nltk
import pandas as pd
import re

from bs4 import BeautifulSoup

def preprocessing(text):
    clean_data = []
    for x in (text[:][0]):
        clean_data =  BeautifulSoup(x, 'html.parser').get_text()
        clean_data = clean_data.lower()#to lower
        clean_data = re.sub(r'[^\w\s]', '', clean_data) # remove punctuation
        #clean_data = nltk.clean_html(clean_data)=>NotImplementedError: To remove HTML markup, use BeautifulSoup's get_text() function
    return clean_data


def preprocessingdf(text):
    #print(text)
    #clean_data = []
    clean_data = text
    clean_data = clean_data.lower()#to lower
    clean_data = BeautifulSoup(clean_data, 'html.parser').get_text()
    return clean_data

test = ['You<p> like &nbsp; to eat apples. He has eaten many apples because he likes eating.']
test_pd = pd.DataFrame(data = test) #makes this into a panda data frame
clean_test = preprocessing(test_pd) #removes punctuation, see above
clean_test

u'you like  to eat apples he has eaten many apples because he likes eating'

In [5]:
data_question['body'][1:9]

1    <p>I accidentally committed the wrong files to...
2    <p>I want to delete a branch both locally and ...
3    <blockquote>\n  <p><strong>Moderator Note:</st...
4    <p>I've been messing around with <a href="http...
5    <p>What is the use of the <code>yield</code> k...
6    <p>After reading <a href="http://groups.google...
7    <p>I mistakenly added files to git using the c...
8    <p>I don't want to rename a remote branch, as ...
Name: body, dtype: object

In [6]:
data_question['body'][1:9].apply(preprocessingdf)
#data_question['clean_body'] = data_question['body'].apply(lambda x : preprocessing(x))


1    i accidentally committed the wrong files to gi...
2    i want to delete a branch both locally and rem...
3    \nmoderator note: given that this question has...
4    i've been messing around with json for some ti...
5    what is the use of the yield keyword in python...
6    after reading hidden features and dark corners...
7    i mistakenly added files to git using the comm...
8    i don't want to rename a remote branch, as des...
Name: body, dtype: object

In [7]:
data_preprocessed = data_question['body'][1:9].apply(tagText.preprocessing)
print(data_preprocessed)

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3    moderator note given question already sixtysev...
4    ive messing around json time pushing text hurt...
5    use yield keyword python do example im trying ...
6    reading hidden features dark corners cstl comp...
7    mistakenly added files git using command git a...
8    want rename remote branch described rename mas...
Name: body, dtype: object


In [8]:
tagText.cleanIngred('i accidentally ')

'accidentally'

In [9]:
data_question.head()

,body,title,tags,SelectedTags,Name
0,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo>,"git,version-control,git-commit,",Question
2,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><git-branch><git-remote>,"git,git-branch,",Question
3,<blockquote>\n <p><strong>Moderator Note:</st...,What is the difference between 'git pull' and ...,<git><git-pull><git-fetch>,"git,",Question
4,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [10]:
data_question.iteritems()

<generator object iteritems at 0x000000001A8DC6C0>

In [11]:
import nltk
from collections import defaultdict
    
corpora = defaultdict(list)
        
tokenizer = nltk.RegexpTokenizer(r'\w+')
for id, row in data_preprocessed.iteritems():
    for word in tokenizer.tokenize(row):
        if not word in corpora :
            corpora[word] = 0
        #print(word)
        corpora[word] += 1
    

#print(corpora)
print(sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

[(u'branch', 16), (u'git', 13), (u'remote', 7), (u'distance', 6), (u'originbugfix', 5), (u'rename', 5), (u'maxdist', 5), (u'local', 4), (u'x', 4), (u'mindist', 3), (u'yield', 3), (u'remotesoriginbugfix', 3), (u'commit', 3), (u'json', 3), (u'files', 3), (u'delete', 3), (u'id', 3), (u'code', 2), (u'yet', 2), (u'candidates', 2), (u'selfmedian', 2), (u'python', 2), (u'know', 2), (u'like', 2), (u'd', 2), (u'called', 2), (u'found', 2), (u'result', 2), (u'error', 2), (u'new', 2), (u'deleted', 2), (u'undo', 2), (u'uptodate', 2), (u'selfleftchild', 2), (u'already', 2), (u'one', 2), (u'locally', 2), (u'question', 2), (u'remotely', 2), (u'type', 2), (u'pull', 2), (u'selfrightchild', 2), (u'int', 2), (u'pushed', 2), (u'want', 2), (u'returned', 2), (u'0', 2), (u'well', 2), (u'consider', 1), (u'text', 1), (u'rest', 1), (u'moderator', 1), (u'attempts', 1), (u'spaces', 1), (u'committed', 1), (u'targeted', 1), (u'issues', 1), (u'rd', 1), (u'jrschulz', 1), (u'c', 1), (u'studio', 1), (u'failed', 1), (u'a

In [12]:
def count_word_occurencies(df):
        corpora = defaultdict(list)                
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for id, row in df.iteritems():
            for word in tokenizer.tokenize(row):
                if not word in corpora :
                    corpora[word] = 0
                #print(word)
                corpora[word] += 1
        
        corpora = dict((k, v) for k, v in corpora.items() if v > 1)
        return (sorted(corpora.items(), reverse=True,  key=lambda x: x[1]))

In [13]:
count_word_occurencies(data_preprocessed)

[(u'branch', 16),
 (u'git', 13),
 (u'remote', 7),
 (u'distance', 6),
 (u'rename', 5),
 (u'maxdist', 5),
 (u'originbugfix', 5),
 (u'x', 4),
 (u'local', 4),
 (u'remotesoriginbugfix', 3),
 (u'id', 3),
 (u'json', 3),
 (u'files', 3),
 (u'mindist', 3),
 (u'yield', 3),
 (u'commit', 3),
 (u'delete', 3),
 (u'code', 2),
 (u'selfrightchild', 2),
 (u'int', 2),
 (u'one', 2),
 (u'pushed', 2),
 (u'already', 2),
 (u'result', 2),
 (u'want', 2),
 (u'yet', 2),
 (u'0', 2),
 (u'locally', 2),
 (u'question', 2),
 (u'remotely', 2),
 (u'selfleftchild', 2),
 (u'deleted', 2),
 (u'candidates', 2),
 (u'new', 2),
 (u'selfmedian', 2),
 (u'returned', 2),
 (u'python', 2),
 (u'undo', 2),
 (u'uptodate', 2),
 (u'know', 2),
 (u'found', 2),
 (u'd', 2),
 (u'pull', 2),
 (u'like', 2),
 (u'type', 2),
 (u'well', 2),
 (u'error', 2),
 (u'called', 2)]

In [14]:
print(tagText.count_word_occurencies(data_preprocessed))

[(u'branch', 16), (u'git', 13), (u'remote', 7), (u'distance', 6), (u'rename', 5), (u'maxdist', 5), (u'originbugfix', 5), (u'x', 4), (u'local', 4), (u'remotesoriginbugfix', 3), (u'id', 3), (u'json', 3), (u'files', 3), (u'mindist', 3), (u'yield', 3), (u'commit', 3), (u'delete', 3), (u'code', 2), (u'selfrightchild', 2), (u'int', 2), (u'one', 2), (u'pushed', 2), (u'already', 2), (u'result', 2), (u'want', 2), (u'yet', 2), (u'0', 2), (u'locally', 2), (u'question', 2), (u'remotely', 2), (u'selfleftchild', 2), (u'deleted', 2), (u'candidates', 2), (u'new', 2), (u'selfmedian', 2), (u'returned', 2), (u'python', 2), (u'undo', 2), (u'uptodate', 2), (u'know', 2), (u'found', 2), (u'd', 2), (u'pull', 2), (u'like', 2), (u'type', 2), (u'well', 2), (u'error', 2), (u'called', 2)]


In [15]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
word_occurence_count = tagText.count_word_occurencies(data_preprocessed)
word_occurence_count_df = pd.DataFrame.from_dict(word_occurence_count).describe()

In [16]:
word_occurence_count_df.head(100)

,1
count,75991.000000
mean,34.263334
std,315.279377
min,2.000000
25%,2.000000
50%,3.000000
75%,6.000000
max,20052.000000


In [17]:
word_occurence_count_df.plot(kind='bar', color="#f56900", title='occurence of word')

In [18]:
dict_tag = count_word_occurencies(data_preprocessed)
dict_tag = str(dict_tag)
dict_tag

"[(u'like', 20052), (u'using', 18156), (u'im', 17409), (u'use', 16357), (u'code', 15098), (u'want', 14713), (u'get', 14451), (u'way', 14095), (u'would', 13548), (u'file', 13163), (u'new', 12636), (u'error', 11625), (u'this', 11259), (u'one', 11081), (u'string', 10799), (u'1', 10739), (u'class', 10381), (u'public', 9653), (u'function', 9527), (u'return', 9350), (u'know', 8714), (u'need', 8502), (u'ive', 7792), (u'0', 7790), (u'following', 7762), (u'something', 7598), (u'example', 7593), (u'it', 7412), (u'2', 7389), (u'set', 7275), (u'c', 7082), (u'data', 7061), (u'work', 6996), (u'id', 6952), (u'method', 6896), (u'value', 6598), (u'var', 6571), (u'find', 6264), (u'name', 6189), (u'trying', 6110), (u'also', 6085), (u'div', 5995), (u'int', 5980), (u'object', 5880), (u'could', 5785), (u'tried', 5660), (u'question', 5639), (u'line', 5498), (u'git', 5486), (u'time', 5468), (u'type', 5426), (u'files', 5418), (u'make', 5360), (u'run', 5257), (u'see', 5207), (u'problem', 5176), (u'list', 5157),

In [19]:
tagText.unsupervised_tag(dict_tag, "like git branche totot, test", 2)

['like', 'git']

In [20]:
def unsupervised_tag(dict_word_key, new_question, number_max_tag):
    tags = [word for word in new_question.split() if (word in dict_word_key)]
    return tags[0:number_max_tag]

print(unsupervised_tag(dict_tag, "like git branche totot, test", 20))
print(unsupervised_tag(dict_tag, "like git branche totot, test", 2))
#for word in "like git branche totot, test".split() if (word in dict_word_key)

['like', 'git', 'branche', 'test']
['like', 'git']


In [21]:
for word in "like git branche totot, test".split() :
    print(word)
    if word in str(dict_tag) :
        print("!!!Yes!!!")

like
!!!Yes!!!
git
!!!Yes!!!
branche
!!!Yes!!!
totot,
test
!!!Yes!!!


In [22]:
print(nltk.word_tokenize(data_question['body'][0])[0:10])
print(nltk.tokenize.WhitespaceTokenizer().tokenize(data_question['body'][0])[0:10])
# Tokenize using Punctuations
print(nltk.tokenize.WordPunctTokenizer().tokenize(data_question['body'][0])[0:10])
print(nltk.tokenize.TreebankWordTokenizer().tokenize(data_question['body'][0])[0:10])

[u'<', u'p', u'>', u'Here', u'is', u'a', u'piece', u'of', u'C++', u'code']
[u'<p>Here', u'is', u'a', u'piece', u'of', u'C++', u'code', u'that', u'shows', u'some']
[u'<', u'p', u'>', u'Here', u'is', u'a', u'piece', u'of', u'C', u'++']
[u'<', u'p', u'>', u'Here', u'is', u'a', u'piece', u'of', u'C++', u'code']


In [23]:

from string import punctuation as ponctuation
ponctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'